In [0]:
# Adapted from
# https://github.com/keras-team/keras/blob/master/examples/addition_rnn.py

In [0]:
import warnings
warnings.filterwarnings('ignore')

In [311]:
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [312]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
print(tf.__version__)

1.9.0-rc0


### Step 1: Generate sample equations

In [0]:
class CharacterTable(object):
    """Given a set of characters:
    + Encode them to a one hot integer representation
    + Decode the one hot integer representation to their character output
    + Decode a vector of probabilities to their character output
    """
    def __init__(self, chars):
        """Initialize character table.

        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One hot encode given string C.

        # Arguments
            num_rows: Number of rows in the returned one hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)


In [0]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [315]:
#@title Model & Dataset Parameters
# Parameters for the model and dataset.
global DIGIT_LEN
global ITERATIONS
global HIDDEN_SIZE
global BATCH_SIZE
global RNN_TYPE
global EPOCHS
RNN_TYPE = "Simple" #@param ["Simple", "GRU", "LSTM"]
TRAINING_SIZE = 50000 #@param {type:"slider", min:100, max:100000, step:100}
EPOCHS = 3 #@param {type:"slider", min:1, max:20, step:1}
ITERATIONS = 20 #@param {type:"slider", min:1, max:40, step:1}
HIDDEN_SIZE = 128 #@param {type:"slider", min:4, max:256, step:4}
ARITH_OPERATION = "ADD" #@param ["ADD", "SUB", "MUL", "DIV"]
OPS = {"ADD":"+", "SUB":"-", "MUL":"*", "DIV":"/"}
OP = OPS[ARITH_OPERATION]
DIGITS = 3
# REVERSE = True
REVERSE = False

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
MAXLEN = DIGITS + 1 + DIGITS

# All the numbers, plus sign and space for padding.

chars = '0123456789' + OP + ' '
ctable = CharacterTable(chars)

questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789'))
                    for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    # Skip any addition questions we've already seen
    # Also skip any such that x+Y == Y+x (hence the sorting).
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    # Pad the data with spaces such that it is always MAXLEN.
    if OP == "/" and b == 0:
      continue
    q = ('{}' + OP + '{}').format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(int(eval(str(a) + OP + str(b))))
    # Answers has a maximum size regarding selected OP
    # => ADD: (DIGITS + 1)
    # => SUB: (DIGITS + 1)
    # => MUL: (DIGITS * 2)
    # => DIV: (DIGITS)
    DIGIT_LENS = {"ADD":DIGITS + 1, "SUB":DIGITS + 1, "MUL":DIGITS * 2, "DIV":DIGITS}
    DIGIT_LEN = DIGIT_LENS[ARITH_OPERATION]
    ans += ' ' * (DIGIT_LEN - len(ans))
    if REVERSE:
        # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
        # space used for padding.)
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

Generating data...
Total addition questions: 50000


In [316]:
questions[0]

'901+335'

In [317]:
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), DIGIT_LEN, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGIT_LEN)


Vectorization...


In [318]:
len(x[0])

7

In [319]:
len(questions[0])

7

In [320]:
questions[0]

'901+335'

In [321]:
x[0]

array([[False, False, False, False, False, False, False, False, False,
        False, False,  True],
       [False, False,  True, False, False, False, False, False, False,
        False, False, False],
       [False, False, False,  True, False, False, False, False, False,
        False, False, False],
       [False,  True, False, False, False, False, False, False, False,
        False, False, False],
       [False, False, False, False, False,  True, False, False, False,
        False, False, False],
       [False, False, False, False, False,  True, False, False, False,
        False, False, False],
       [False, False, False, False, False, False, False,  True, False,
        False, False, False]])

In [322]:
y[0]

array([[False, False, False,  True, False, False, False, False, False,
        False, False, False],
       [False, False, False, False,  True, False, False, False, False,
        False, False, False],
       [False, False, False, False, False,  True, False, False, False,
        False, False, False],
       [False, False, False, False, False, False, False, False,  True,
        False, False, False]])

In [323]:
expected[0]

'1236'

In [0]:
# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

### Step 2: Training/Validation Split

In [325]:
# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

Training Data:
(45000, 7, 12)
(45000, 4, 12)
Validation Data:
(5000, 7, 12)
(5000, 4, 12)


### Step 3: Create Model

In [326]:
# input shape: 7 digits, each being 0-9, + or space (12 possibilities)
MAXLEN, len(chars)

(7, 12)

In [327]:
from keras.models import Sequential
from keras import layers

# Select RNN Type by param RNN_TYPE
RNN = {"Simple":layers.SimpleRNN, "GRU":layers.GRU, "LSTM":layers.LSTM}[RNN_TYPE]

#HIDDEN_SIZE = 128
BATCH_SIZE = 128

print('Build model...')
model = Sequential()
# encoder: simply encode the same input 4 times 
model.add(RNN(units=HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
model.add(layers.RepeatVector(DIGIT_LEN))

# decoder: have 4 temporal outputs one for each of the digits of the results
# return_sequences=True tells it to keep all 4 temporal outputs, not only the final one (we need all four digits for the results)
model.add(RNN(units=HIDDEN_SIZE, return_sequences=True))

model.add(layers.Dense(name='classifier', units=len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_21 (SimpleRNN)    (None, 128)               18048     
_________________________________________________________________
repeat_vector_11 (RepeatVect (None, 4, 128)            0         
_________________________________________________________________
simple_rnn_22 (SimpleRNN)    (None, 4, 128)            32896     
_________________________________________________________________
classifier (Dense)           (None, 4, 12)             1548      
Total params: 52,492
Trainable params: 52,492
Non-trainable params: 0
_________________________________________________________________


In [328]:
model.predict(np.array([x_val[0]]))

array([[[0.11871254, 0.04725668, 0.08597811, 0.06052801, 0.04916698,
         0.10806846, 0.0652658 , 0.09213211, 0.07725427, 0.06431834,
         0.13919382, 0.09212484],
        [0.09019678, 0.06710916, 0.0778727 , 0.0774928 , 0.06764337,
         0.12845382, 0.05008734, 0.06974961, 0.08910798, 0.07944917,
         0.127128  , 0.07570926],
        [0.107257  , 0.06185251, 0.07222693, 0.1455909 , 0.05805711,
         0.09302455, 0.05640073, 0.05220547, 0.06964339, 0.08810502,
         0.12289277, 0.07274362],
        [0.07308451, 0.07377408, 0.07277254, 0.08005625, 0.07884493,
         0.0896541 , 0.05537273, 0.04966318, 0.07782666, 0.12280977,
         0.14085534, 0.08528587]]], dtype=float32)

In [329]:
model.predict_classes(np.array([x_val[0]]))

array([[10,  5,  3, 10]])

### Step 4: Train

In [0]:
# Train the model each generation and show predictions against the validation
# dataset.
for iteration in range(1, ITERATIONS + 1):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=EPOCHS,
              validation_data=(x_val, y_val))
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 1
Train on 45000 samples, validate on 5000 samples
Epoch 1/3
43520/45000 [============================>.] - ETA: 0s - loss: 1.6703 - acc: 0.3911

In [0]:
x_val[0]

In [0]:
model.predict(np.array([x_val[0]]))

In [0]:
model.predict_classes(np.array([x_val[0]]))